# Summary

We're going to derive logistic regression then run some simulations.

## References

* [Strang, Introduction to Linear Algebra](https://math.mit.edu/~gs/linearalgebra/)

In [1]:
import numpy as np

np.random.seed(42)

# Strang 4.2 Projections

Starting with matrix projections. Reviewing Example 3 (p. 211)

In [2]:
A = np.array([[1,0],[1,1],[1,2]]); b = np.reshape(np.array([6,0,0]),(-1,1))


In [3]:
# A^T A
np.dot(np.transpose(A),A)

array([[3, 3],
       [3, 5]])

In [4]:
# A^T b
np.dot(np.transpose(A),b)

array([[6],
       [0]])

In [5]:
P = np.dot(np.linalg.inv(np.dot(np.transpose(A),A)),np.transpose(A)) # (7) projection matrix

In [6]:
# Now solve the normal equation A^T * A*xhat = A^T * bhat

xhat = np.dot(P,b) # (8)
xhat

array([[ 5.],
       [-3.]])

In [7]:
# The combination p = A * xhat is the projection of b onto the column space of A
p = np.dot(A,xhat)
p

array([[ 5.],
       [ 2.],
       [-1.]])

In [8]:
# The error is
e = b - p
e # (9)

array([[ 1.],
       [-2.],
       [ 1.]])

In [9]:
# Two checks on the calculation. 
all(# First, the error is perpendicular to both columns.
    [np.dot(A[:,0],e) == 0, np.dot(A[:,1],e) == 0],
    # Second, the matrix P times b correctly gives p
)

True

# Strang 4.3 Least Squares Approximations

When $Ax=b$ has no solution, $\hat{x}$ is the "least-squares solution": $||b - A \hat{x}||^2 = $ minimum.

## Example 1

A crucial application of least squares is fitting a straight line to $m$ points. This 3 by 2 system has *no solution*.

In [10]:
A = np.array([[1,0],[1,1],[1,2]]); b = np.reshape(np.array([6,0,0]),(-1,1))
A, b # Ax = b is not solvable

(array([[1, 0],
        [1, 1],
        [1, 2]]),
 array([[6],
        [0],
        [0]]))

## Minimizing the Error

The best $\hat{x}$ comes from the normal equations $A^TA \hat{x} = A^Tb$. $E$ is a minimum.

In [11]:
xhat = np.dot(np.linalg.inv(np.dot(np.transpose(A),A)), np.dot(np.transpose(A),b))
xhat # (1)

array([[ 5.],
       [-3.]])

In [12]:
p = np.dot(A,xhat) # (2)
p

array([[ 5.],
       [ 2.],
       [-1.]])

In [13]:
e = b - p
all([np.sum(e) == 0])

True

# Generating a Random Matrix

In [14]:
# X is the features
m = 100; n = 10
X = np.random.rand(m,n)

# set the first attribute to one for an intercept
X[:,1] = 1

# y is the target
y = np.random.rand(m,1)

print("X shape {}, y shape {}".format(X.shape, y.shape))

X shape (100, 10), y shape (100, 1)


In [15]:
y_cls = np.reshape(np.random.randint(low=0,high=2,size=m),(-1,1))
y_cls.shape

(100, 1)

In [16]:
np.sum(y_cls) / y_cls.shape[0]

0.47

Convert train/test to 80/20%.

In [17]:
split = 0.8
m_train = int(m * split)
m_train

80

In [18]:
X.shape

(100, 10)

In [19]:
X_train = X[:m_train,:]; y_train = y[:m_train]; y_cls_train = y_cls[:m_train]
X_train.shape, y_train.shape, y_cls_train.shape

((80, 10), (80, 1), (80, 1))

In [20]:
X_test = X[m_train:,:]; y_test = y[m_train:]; y_cls_test = y_cls[m_train:]
X_test.shape, y_test.shape, y_cls_test.shape

((20, 10), (20, 1), (20, 1))

# Least Squares Approximation

In [21]:
# Analytic solution to linear regression
# 
# Strang Ch. 4

def analyticlm(A,b):
    xhat = np.dot(np.linalg.inv(np.dot(np.transpose(A),A)), np.dot(np.transpose(A),b))
    return  xhat

In [22]:
bhat = analyticlm(X_train,y_train)
bhat

array([[ 0.20549593],
       [ 0.34878373],
       [ 0.17357996],
       [ 0.08132801],
       [ 0.00414967],
       [-0.01918945],
       [ 0.0961147 ],
       [-0.11891359],
       [ 0.09076461],
       [-0.10092084]])

In [23]:
predict = np.dot(X_test,bhat)
predict

array([[0.64035738],
       [0.62187278],
       [0.60194866],
       [0.46148819],
       [0.32835673],
       [0.46124611],
       [0.60196935],
       [0.53815127],
       [0.42695011],
       [0.58659693],
       [0.47818277],
       [0.57633164],
       [0.741798  ],
       [0.43208381],
       [0.51146044],
       [0.55628168],
       [0.47156483],
       [0.66080177],
       [0.38975595],
       [0.72117655]])

In [24]:
def evaluate_ols(pred, actual):
    """ Root-Mean Square Error. 
    
    Perfect score would be 0.0.
    """
    assert pred.shape == actual.shape
    return np.sqrt(np.sum((pred - actual)**2) / pred.shape[0])

In [25]:
evaluate_ols(predict, y_test)

0.3550899427668016

# Logistic Regression with a Random Matrix

Sigmoid function is $S(x) = \frac{1}{1 + \exp^{-x}}$

In [26]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [27]:
def analytic_logit(A,b):
    xhat = analyticlm(A,b)
    return sigmoid(xhat)

In [28]:
bhat = analytic_logit(X_train,y_cls_train)
bhat

array([[0.50223367],
       [0.67298951],
       [0.53451623],
       [0.41897932],
       [0.48359603],
       [0.54483137],
       [0.43374557],
       [0.50159306],
       [0.4783385 ],
       [0.4579651 ]])

In [29]:
# https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html#sigmoid-activation

predictions = sigmoid(np.dot(X_test,bhat))
predictions.shape

(20, 1)

In [30]:
def decision_boundary(prob, boundary=0.5):
    return np.where(prob >= boundary, 1,0)

In [31]:
def accuracy(predicted_labels, actual_labels):
    diff = predicted_labels - actual_labels
    return 1.0 - (float(np.count_nonzero(diff)) / len(diff))

In [32]:
predicted_labels = decision_boundary(predictions)
predicted_labels

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [33]:
accuracy(predicted_labels, y_cls_test)

0.55

# Class Imbalance for Logistic Regression on a Random Matrix

Let's assume 90/10%

In [34]:
np.sum(y_cls_train) / y_cls_train.shape[0]

0.45

In [35]:
y_cls_imb_train = y_cls_train
y_cls_imb_train.shape

(80, 1)

In [36]:
y_cls_imb_train = y_cls_imb_train * 0
y_cls_imb_train[:int(y_cls_imb_train.shape[0] * 0.9)] = 1

np.random.shuffle(y_cls_imb_train)

In [37]:
y_cls_imb_test = y_cls_test

y_cls_imb_test = y_cls_imb_test * 0
y_cls_imb_test[:int(y_cls_imb_test.shape[0] * 0.9)] = 1

np.random.shuffle(y_cls_imb_test)

In [38]:
# Compute coefficients

bhat_imb = analytic_logit(X_train, y_cls_imb_train)
bhat_imb

array([[0.52990053],
       [0.69149814],
       [0.52803463],
       [0.5371479 ],
       [0.48183059],
       [0.4749205 ],
       [0.51548101],
       [0.48592297],
       [0.48988449],
       [0.51198536]])

In [39]:
predictions = sigmoid(np.dot(X_test,bhat_imb))

In [40]:
predicted_labels = decision_boundary(predictions)
predicted_labels

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [41]:
accuracy(predicted_labels, y_cls_imb_test)

0.9